# Data 608 Assignment 4

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd
import numpy as np
import plotly.graph_objects as go

## Data Pull

In [2]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id),boroname,health,steward' +\
        '&$group=spc_common,boroname,health,steward').replace(' ', '%20')

page1 = pd.read_json(soql_url)
page2 = pd.read_json(soql_url + '&$offset=1000')
page3 = pd.read_json(soql_url + '&$offset=2000')
page4 = pd.read_json(soql_url + '&$offset=3000')
page5 = pd.read_json(soql_url + '&$offset=4000')

trees = pd.concat([page1, page2, page3, page4, page5])
trees

,spc_common,count_tree_id,boroname,health,steward
0,American beech,3,Bronx,Fair,1or2
1,American beech,1,Bronx,Fair,3or4
2,American beech,3,Bronx,Fair,None
3,American beech,2,Bronx,Good,1or2
4,American beech,1,Bronx,Good,3or4
...,...,...,...,...,...
560,NaN,2,Queens,Good,1or2
561,NaN,1,Queens,Good,None
562,NaN,1,Queens,Poor,None
563,NaN,12577,Queens,NaN,NaN


In [3]:
health = trees[trees['health'].notna()]
health = trees[trees['spc_common'].notna()]
health

,spc_common,count_tree_id,boroname,health,steward
0,American beech,3,Bronx,Fair,1or2
1,American beech,1,Bronx,Fair,3or4
2,American beech,3,Bronx,Fair,None
3,American beech,2,Bronx,Good,1or2
4,American beech,1,Bronx,Good,3or4
...,...,...,...,...,...
551,willow oak,29,Staten Island,Good,1or2
552,willow oak,4,Staten Island,Good,3or4
553,willow oak,177,Staten Island,Good,None
554,willow oak,2,Staten Island,Poor,1or2


## Questions 1 & 2

In [4]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("NYC Tree Health"),
    dcc.Graph(id='graph'),
    html.Label([
        "Species Filter",
        dcc.Dropdown(
        id='species',
         options=[
        {'label': b, 'value': b} for b in health.spc_common.unique()],
        value='American beech', multi=False, placeholder='Filter by Species'),
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("species", "value")])
def update_figure(species_value):
    df = health[(health['spc_common']==species_value)]
    return px.bar(
        df, x="health", y="count_tree_id", color='boroname', title="NYC Tree Health by Borough & Species"
    )

In [ ]:
app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)
